In [2]:
from llama_index.llms.ollama import Ollama

# Initialize the first model
llm1 = Ollama(model="qwen2.5:72b", request_timeout=120.0)

# Initialize the second model
llm2 = Ollama(model="deepseek-r1:70b", request_timeout=120.0)


In [3]:
def query_models(prompt):
    # Query the first model
    response1 = llm1.complete(prompt)

    # Query the second model
    response2 = llm2.complete(prompt)

    return response1, response2


In [ ]:
prompt = "Translate the following English text to French: 'Hello, how are you?'"
response1, response2 = query_models(prompt)

print("Response from Model 1:", response1)
print("Response from Model 2:", response2)


In [31]:
from llama_index.llms.ollama import Ollama

# Initialize the first model
llm1 = Ollama(model="qwen2.5:72b", request_timeout=120.0)
llm2 = Ollama(model="llama3.3:latest", request_timeout=120.0)

In [13]:
from llama_index.core.tools import BaseTool, FunctionTool
_memory = "user name is abc"

def current_memory_function() -> str:
    """Returns the current global memory string.
    
    Returns:
        str: The accumulated memory string (editable globally), 
             initially empty
    """
    return _memory
current_memory_function_tool = FunctionTool.from_defaults(fn=current_memory_function)

def update_memory(new_info: str) -> None:
    """Updates the global memory with new information if not already present.
    
    Args:
        new_info (str): The string to add to memory if it contains 
                        previously unknown information
    """
    global _memory
    entries = _memory.split('\n') if _memory else []
    
    # Add only if new information not in existing memory
    if new_info.strip() and new_info not in entries:
        entries.append(new_info)
        _memory = '\n'.join(entries)
update_memory_tool = FunctionTool.from_defaults(fn=update_memory)        

In [14]:
from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools([current_memory_function_tool, update_memory_tool], llm=llm1, verbose=True)

In [ ]:
response = agent.chat("What do u know so far")
print(response)
response = agent.chat("i want to teach you something , i love to be happy")
print(response)
response = agent.chat("i love sushi")
print(response)

In [ ]:
response = agent.chat("What do u know so far")
print(response)

sql engine

In [1]:
from llama_index.core import SQLDatabase
from sqlalchemy import create_engine
from dataclasses import dataclass, field

# Configuration for the database
@dataclass
class Config:
    DB_HOST: str = "143.198.230.83"
    DB_PORT: int = 3306
    DB_NAME: str = "golgixportal"
    DB_USER: str = "golgix"
    DB_PASSWORD: str = "preciseV5"
    ALLOWED_TABLES: list[str] = field(default_factory=lambda: [
        "DE",
        "HPLC_Data",
        "fermentation_data",
        "plc_data_1",
        "plc_data_2",
        "plc_data_3",
        "plc_data_4"
    ])

# Load configuration
config = Config()

# Setup the SQLAlchemy engine URI
db_uri = f"mysql+pymysql://{config.DB_USER}:{config.DB_PASSWORD}@{config.DB_HOST}:{config.DB_PORT}/{config.DB_NAME}"

# Initialize the SQLDatabase with the URI and include_tables parameter
sql_database = SQLDatabase.from_uri(db_uri, include_tables=config.ALLOWED_TABLES)

In [3]:
from llama_index.llms.ollama import Ollama

# Initialize the first model
llm1 = Ollama(model="qwen2.5:72b", request_timeout=120.0)

In [4]:
from llama_index.core.query_pipeline import QueryPipeline

In [5]:
# setup Arize Phoenix for logging/observability
import phoenix as px
import llama_index.core

px.launch_app()
llama_index.core.set_global_handler("arize_phoenix")

/home/sagar/anaconda3/envs/factfinder/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [6]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.tools import QueryEngineTool

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    llm=llm1,
    embed_model='local',
    # tables=["albums", "tracks", "artists"],
    verbose=True,

)
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=(
        "Useful for translating a natural language query into a SQL query"
    ),
)

In [7]:
from llama_index.core.query_pipeline import QueryPipeline as QP

qp = QP(verbose=True)

In [8]:
from llama_index.core.agent.react.types import (
    ActionReasoningStep,
    ObservationReasoningStep,
    ResponseReasoningStep,
)
from llama_index.core.agent import Task, AgentChatResponse
from llama_index.core.query_pipeline import (
    StatefulFnComponent,
    QueryComponent,
    ToolRunnerComponent,
)
from llama_index.core.llms import MessageRole
from typing import Dict, Any, Optional, Tuple, List, cast


# Input Component
## This is the component that produces agent inputs to the rest of the components
## Can also put initialization logic here.
def agent_input_fn(state: Dict[str, Any]) -> str:
    """Agent input function.

    Returns:
        A Dictionary of output keys and values. If you are specifying
        src_key when defining links between this component and other
        components, make sure the src_key matches the specified output_key.

    """
    task = state["task"]
    if len(state["current_reasoning"]) == 0:
        reasoning_step = ObservationReasoningStep(observation=task.input)
        state["current_reasoning"].append(reasoning_step)
    return task.input


agent_input_component = StatefulFnComponent(fn=agent_input_fn)

In [9]:
from llama_index.core.agent import ReActChatFormatter
from llama_index.core.query_pipeline import InputComponent, Link
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool


## define prompt function
def react_prompt_fn(
    state: Dict[str, Any], input: str, tools: List[BaseTool]
) -> List[ChatMessage]:
    task = state["task"]
    # Add input to reasoning
    chat_formatter = ReActChatFormatter()
    cur_prompt = chat_formatter.format(
        tools,
        chat_history=task.memory.get(),
        current_reasoning=state["current_reasoning"],
    )
    return cur_prompt


react_prompt_component = StatefulFnComponent(
    fn=react_prompt_fn, partial_dict={"tools": [sql_tool]}
)

In [10]:
from typing import Set, Optional
from llama_index.core.agent.react.output_parser import ReActOutputParser
from llama_index.core.llms import ChatResponse
from llama_index.core.agent.types import Task


def parse_react_output_fn(state: Dict[str, Any], chat_response: ChatResponse):
    """Parse ReAct output into a reasoning step."""
    output_parser = ReActOutputParser()
    reasoning_step = output_parser.parse(chat_response.message.content)
    return {"done": reasoning_step.is_done, "reasoning_step": reasoning_step}


parse_react_output = StatefulFnComponent(fn=parse_react_output_fn)


def run_tool_fn(state: Dict[str, Any], reasoning_step: ActionReasoningStep):
    """Run tool and process tool output."""
    task = state["task"]
    tool_runner_component = ToolRunnerComponent(
        [sql_tool], callback_manager=task.callback_manager
    )
    tool_output = tool_runner_component.run_component(
        tool_name=reasoning_step.action,
        tool_input=reasoning_step.action_input,
    )
    observation_step = ObservationReasoningStep(observation=str(tool_output))
    state["current_reasoning"].append(observation_step)
    # TODO: get output

    # return tuple of current output and False for is_done
    return observation_step.get_content(), False


run_tool = StatefulFnComponent(fn=run_tool_fn)


def process_response_fn(
    state: Dict[str, Any], response_step: ResponseReasoningStep
):
    """Process response."""
    state["current_reasoning"].append(response_step)
    return response_step.response, True


process_response = StatefulFnComponent(fn=process_response_fn)

In [11]:
from llama_index.core.query_pipeline import QueryPipeline as QP
from llama_index.llms.openai import OpenAI

qp.add_modules(
    {
        "agent_input": agent_input_component,
        "react_prompt": react_prompt_component,
        "llm": llm1,
        "react_output_parser": parse_react_output,
        "run_tool": run_tool,
        "process_response": process_response,
    }
)

In [12]:
# link input to react prompt to parsed out response (either tool action/input or observation)
qp.add_chain(["agent_input", "react_prompt", "llm", "react_output_parser"])

# add conditional link from react output to tool call (if not done)
qp.add_link(
    "react_output_parser",
    "run_tool",
    condition_fn=lambda x: not x["done"],
    input_fn=lambda x: x["reasoning_step"],
)
# add conditional link from react output to final response processing (if done)
qp.add_link(
    "react_output_parser",
    "process_response",
    condition_fn=lambda x: x["done"],
    input_fn=lambda x: x["reasoning_step"],
)

In [13]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(qp.clean_dag)
net.show("agent_dag.html")

agent_dag.html


In [14]:
from llama_index.core.agent import FnAgentWorker
from typing import Dict, Tuple, Any


def run_agent_fn(state: Dict[str, Any]) -> Tuple[Dict[str, Any], bool]:
    """Run agent function."""
    task, qp = state["__task__"], state["query_pipeline"]
    # if first run, then set query pipeline state to initial variables
    if state["is_first"]:
        qp.set_state(
            {
                "task": task,
                "current_reasoning": [],
            }
        )
        state["is_first"] = False

    # no explicit input here, just run root node
    response_str, is_done = qp.run()
    # if done, store output and log to memory
    # a core memory module is available in the `task` variable. Of course you can log
    # and store your own memory as well
    state["__output__"] = response_str
    if is_done:
        task.memory.put_messages(
            [
                ChatMessage(content=task.input, role=MessageRole.USER),
                ChatMessage(content=response_str, role=MessageRole.ASSISTANT),
            ]
        )
    return state, is_done


agent = FnAgentWorker(
    fn=run_agent_fn,
    initial_state={"query_pipeline": qp, "is_first": True},
).as_agent()

In [15]:
# start task
task = agent.create_task(
    "Downtime on equipment last 24 hours"
)

In [ ]:
step_output = agent.run_step(task.task_id)
step_output = agent.run_step(task.task_id)

> Running module agent_input with input: 

> Running module react_prompt with input: 
input: Downtime on equipment last 24 hours

> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='You are designed to help with a variety of tasks, from answering questions to providi...

> Running module react_output_parser with input: 
chat_response: assistant: Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: In the last 24 hours, there has been no downtime on the equipment. The query did not retu...

> Running module process_response with input: 
response_step: thought="I can answer without using any more tools. I'll use the user's language to answer." response='In the last 24 hours, there has been no downtime on the equipment. The query did not return any r...



In [20]:
step_output.is_last

True

In [22]:
response = agent.finalize_response(task.task_id)
print(str(response))

In the last 24 hours, there has been no downtime on the equipment. The query did not return any records where the flow rate (FI_3150_PV) was zero, indicating that the equipment has been operational throughout this period.


## Setup Simple Retry Agent Pipeline for Text-to-SQL 

let's try a much simpler agent pipeline that only does text-to-SQL, with retry-logic.

We try a simple text-based "retry" prompt where given the user input and previous conversation history, can generate a modified query that outputs the right result.

In [27]:
from llama_index.llms.ollama import Ollama

# Initialize the first model
llm1 = Ollama(model="qwen2.5:72b", request_timeout=300.0)
# llm1 = Ollama(model="qwen2.5:72b")

In [28]:
from llama_index.core import SQLDatabase
from sqlalchemy import create_engine
from dataclasses import dataclass, field

# Configuration for the database
@dataclass
class Config:
    DB_HOST: str = "143.198.230.83"
    DB_PORT: int = 3306
    DB_NAME: str = "golgixportal"
    DB_USER: str = "golgix"
    DB_PASSWORD: str = "preciseV5"
    ALLOWED_TABLES: list[str] = field(default_factory=lambda: [
        "DE",
        "HPLC_Data",
        "fermentation_data",
        "plc_data_1",
        "plc_data_2",
        "plc_data_3",
        "plc_data_4"
    ])

# Load configuration
config = Config()

# Setup the SQLAlchemy engine URI
db_uri = f"mysql+pymysql://{config.DB_USER}:{config.DB_PASSWORD}@{config.DB_HOST}:{config.DB_PORT}/{config.DB_NAME}"

# Initialize the SQLDatabase with the URI and include_tables parameter
sql_database = SQLDatabase.from_uri(db_uri, include_tables=config.ALLOWED_TABLES)

In [29]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.tools import QueryEngineTool

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    llm=llm1,
    embed_model='local',
    # tables=["albums", "tracks", "artists"],
    verbose=True,

)
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=(
        "Useful for translating a natural language query into a SQL query"
    ),
)

In [30]:
from llama_index.core.agent import Task, AgentChatResponse
from typing import Dict, Any
from llama_index.core.query_pipeline import StatefulFnComponent


def agent_input_fn(state: Dict[str, Any]) -> Dict:
    """Agent input function."""
    task = state["task"]
    state["convo_history"].append(f"User: {task.input}")
    convo_history_str = "\n".join(state["convo_history"]) or "None"
    return {"input": task.input, "convo_history": convo_history_str}


agent_input_component = StatefulFnComponent(fn=agent_input_fn)

In [31]:
from llama_index.core import PromptTemplate

retry_prompt_str = """\
You are trying to generate a proper natural language query given a user input.

This query will then be interpreted by a downstream text-to-SQL agent which
will convert the query to a SQL statement. If the agent triggers an error,
then that will be reflected in the current conversation history (see below).

If the conversation history is None, use the user input. If its not None,
generate a new SQL query that avoids the problems of the previous SQL query.

Input: {input}
Convo history (failed attempts): 
{convo_history}

New input: """
retry_prompt = PromptTemplate(retry_prompt_str)

In [32]:
from llama_index.core import Response
from typing import Tuple

validate_prompt_str = """\
Given the user query, validate whether the inferred SQL query and response from executing the query is correct and answers the query.

Answer with YES or NO.

Query: {input}
Inferred SQL query: {sql_query}
SQL Response: {sql_response}

Result: """
validate_prompt = PromptTemplate(validate_prompt_str)

MAX_ITER = 5


def agent_output_fn(
    state: Dict[str, Any], output: Response
) -> Tuple[AgentChatResponse, bool]:
    """Agent output component."""
    task = state["task"]
    print(f"> Inferred SQL Query: {output.metadata['sql_query']}")
    print(f"> SQL Response: {str(output)}")
    state["convo_history"].append(
        f"Assistant (inferred SQL query): {output.metadata['sql_query']}"
    )
    state["convo_history"].append(f"Assistant (response): {str(output)}")

    # run a mini chain to get response
    validate_prompt_partial = validate_prompt.as_query_component(
        partial={
            "sql_query": output.metadata["sql_query"],
            "sql_response": str(output),
        }
    )
    qp = QP(chain=[validate_prompt_partial, llm1])
    validate_output = qp.run(input=task.input)

    state["count"] += 1
    is_done = False
    if state["count"] >= MAX_ITER:
        is_done = True
    if "YES" in validate_output.message.content:
        is_done = True

    return str(output), is_done


agent_output_component = StatefulFnComponent(fn=agent_output_fn)

In [33]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)

qp = QP(
    modules={
        "input": agent_input_component,
        "retry_prompt": retry_prompt,
        "llm": llm1,
        "sql_query_engine": sql_query_engine,
        "output_component": agent_output_component,
    },
    verbose=True,
)
qp.add_link(
    "input", "retry_prompt", dest_key="input", input_fn=lambda x: x["input"]
)
qp.add_link(
    "input",
    "retry_prompt",
    dest_key="convo_history",
    input_fn=lambda x: x["convo_history"],
)
qp.add_chain(["retry_prompt", "llm", "sql_query_engine", "output_component"])

In [34]:
from llama_index.core.agent import FnAgentWorker


def run_agent_fn(state: Dict[str, Any]) -> Tuple[Dict[str, Any], bool]:
    """Run agent function."""
    task, qp = state["__task__"], state["query_pipeline"]
    # if first run, then set query pipeline state to initial variables
    if state["is_first"]:
        qp.set_state({"task": task, "convo_history": [], "count": 0})
        state["is_first"] = False

    # run the pipeline, get response
    response_str, is_done = qp.run()
    if is_done:
        state["__output__"] = response_str
    return state, is_done


agent = FnAgentWorker(
    fn=run_agent_fn,
    initial_state={"query_pipeline": qp, "is_first": True},
).as_agent()

In [35]:
response = agent.chat(
    "Downtime on equipment last 24 hours ?"
)
print(str(response))

> Running module input with input: 

> Running module retry_prompt with input: 
input: Downtime on equipment last 24 hours ?
convo_history: User: Downtime on equipment last 24 hours ?

> Running module llm with input: 
messages: You are trying to generate a proper natural language query given a user input.

This query will then be interpreted by a downstream text-to-SQL agent which
will convert the query to a SQL statement. I...

> Running module sql_query_engine with input: 
input: assistant: Given the user's request and the conversation history, it seems like the initial attempt to generate a SQL query for "Downtime on equipment last 24 hours" might have failed. To refine the q...

> Table Info: Table 'DE' has columns: recordid (INTEGER), datetime (DATETIME), date (DATE), shift (VARCHAR(50)), operator (VARCHAR(50)), hour (VARCHAR(100)), category (VARCHAR(50)), parameter (VARCHAR(50)), target (VARCHAR(50)), entered_value (VARCHAR(50)), user (VARCHAR(50)), .
> Table Info: Table 'HPLC_D